In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#for ML preprocessing
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
#neural network
from tensorflow import keras
from tensorflow.keras import layers
from keras import optimizers
#nerual net analysis
import shap
#random forest classifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV #for gridsearch of rf
#evaluation
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

#Neural Net Analysis
import shap

import MLACDS as dt #diffusion simulation tools

pd.set_option('display.max_columns', None)

2024-02-01 14:27:04.157662: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


Files with useable data:
'p0.1_N54/'
'p0.2_N54/'
'p0.3_N500/'

In [2]:
N_neighbours = 8 ### number of neighbours to be extracted

# dires = ["p0.1_N54/","p0.2_N54/"] #data we want to load for training
# dires = ["p0.3_N500/"] #not compatible with above two due to different particles

dires = ['p0.1_N54/', 'p0.2_N54/'] 

In [3]:
#### list with names to be used in the final dataframe
l_s = list(range(N_neighbours))
l_s_names = []


for i in l_s:
    l_s_names.append('l'+str(i)+"_inv")

for i in l_s:
    l_s_names.append('l'+str(i))


In [4]:
l_s_names

['l0_inv',
 'l1_inv',
 'l2_inv',
 'l3_inv',
 'l4_inv',
 'l5_inv',
 'l6_inv',
 'l7_inv',
 'l0',
 'l1',
 'l2',
 'l3',
 'l4',
 'l5',
 'l6',
 'l7']

In [5]:
### list of columns for bond order parameter
bo = list(range(N_neighbours))
bo_names = []
for i in bo:
    bo_names.append("bo"+str(i))

In [6]:
f_r_cut_vol = 4### cut off used for the local volume calculation
# radius of local volume spher 
#keep @ 4 too small too little neighbour

f_r_cut_sp = 30###A big number here means look for the closest 
                #neighbours
               ###For a small cut-off the number of neighbours
                #might not be the same for all particles


In [7]:
df = dt.load_data(dires, l_s, l_s_names, bo, bo_names, f_r_cut_vol, f_r_cut_sp, N_neighbours)
# df = pd.read_csv("data/csv_files/diffusion_data.csv")

  0%|          | 0/200 [00:00<?, ?it/s]

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


(54, 8)
(54, 8)


  0%|          | 0/100 [00:00<?, ?it/s]

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


(54, 8)
(54, 8)


In [23]:
df.to_csv("data/csv_files/diffusion_data.csv", index=False)

In [22]:
df

,l0_inv,l1_inv,l2_inv,l3_inv,l4_inv,l5_inv,l6_inv,l7_inv,l0,l1,l2,l3,l4,l5,l6,l7,bo0,bo1,bo2,bo3,bo4,bo5,bo6,bo7,vol,n_neighbours,Ds
0,0.506456,0.509452,0.580007,0.621396,0.640808,0.708071,0.774553,0.808400,4.323711,4.346685,4.354562,4.405185,4.406507,4.452742,4.834611,4.988064,3.544908,4.252185e-17,0.059941,4.306939e-17,0.275870,4.589469e-17,0.112218,6.919064e-17,0.139783,53.0,0.521434
1,0.515427,0.533557,0.550062,0.579262,0.594445,0.612513,0.654626,0.837130,3.961337,4.321118,4.366739,4.446013,4.475399,4.543110,4.618340,4.669307,3.544908,4.899669e-17,0.059941,4.131960e-17,0.275870,4.556318e-17,0.112218,6.990912e-17,0.142876,53.0,0.532875
2,0.471929,0.480448,0.512782,0.523910,0.554297,0.654626,0.910404,0.919762,4.191260,4.321971,4.349538,4.500502,4.530259,4.554309,4.557022,4.724020,3.544908,4.805159e-17,0.059941,3.859335e-17,0.275870,4.583321e-17,0.112218,7.051412e-17,0.106246,53.0,0.485428
3,0.507962,0.514168,0.515427,0.554517,0.655009,0.751972,0.810641,0.910404,4.189346,4.192980,4.236030,4.277729,4.361502,4.524403,4.572871,5.158310,3.544908,5.356737e-17,0.059941,4.203535e-17,0.275870,4.454313e-17,0.112218,6.885246e-17,0.119037,53.0,0.492425
4,0.415394,0.448175,0.448297,0.474437,0.505036,0.556669,0.671539,0.810641,4.056734,4.074428,4.298979,4.428562,4.437737,4.437862,4.557605,4.771136,3.544908,4.933025e-17,0.059941,3.891026e-17,0.275870,4.468783e-17,0.112218,6.982776e-17,0.075929,53.0,0.562989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16195,0.614620,0.621393,0.658428,0.676909,0.698650,0.743611,0.803709,0.857511,3.296563,3.440673,3.538916,3.693100,3.724791,3.726209,3.762928,3.797280,3.544908,3.059538e-17,0.034166,3.328186e-17,0.253684,4.976994e-17,0.078902,7.386497e-17,0.215685,53.0,0.393998
16196,0.544488,0.611636,0.633573,0.705376,0.734750,0.821348,0.857511,0.864222,3.336692,3.354459,3.393449,3.495776,3.515479,3.538961,3.741113,3.783643,3.544908,3.030363e-17,0.034166,3.317297e-17,0.253684,4.979619e-17,0.078902,7.383386e-17,0.198661,53.0,0.386789
16197,0.614163,0.670857,0.734750,0.767399,0.781626,0.794275,0.840062,0.852348,3.219074,3.237361,3.249263,3.304907,3.396367,3.483600,3.690005,3.949923,3.544908,2.992687e-17,0.034166,3.313902e-17,0.253684,4.981818e-17,0.078902,7.372116e-17,0.201818,53.0,0.374940
16198,0.587466,0.591041,0.593595,0.608679,0.670857,0.729277,0.758259,0.796437,3.358284,3.359473,3.398156,3.624292,3.647157,3.704437,3.777765,3.882116,3.544908,3.025256e-17,0.034166,3.306143e-17,0.253684,4.980164e-17,0.078902,7.381448e-17,0.238355,53.0,0.415230


## Data Preprocessing

In [11]:
data = df.iloc[:,:-1]

In [12]:
target = df.Ds

In [13]:
#splitting the data to train and test
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=13, test_size=0.4)

In [14]:
#scaling data based on training set to prevent data leakage, the scaler computed from training set statistics will be applied to test data
scaler = preprocessing.StandardScaler().fit(X_train)

In [15]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## NN Model

In [16]:
#the first nn model. This contains:
model = keras.Sequential([
    layers.Dense(32, input_shape=(18,), activation="relu"), #input layer
    layers.Dense(16, activation="relu"), #first hidden layer less nodes
    layers.Dropout(0.5), #dropout layer with 50% dropout to prevent overfitting
    layers.Dense(1) #output layer, no activation function as must be continuous
])

2024-02-01 14:28:42.434186: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                608       
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1,153
Trainable params: 1,153
Non-trainable params: 0
_________________________________________________________________


In [18]:
# # Compile the model
model.compile(optimizer=optimizers.RMSprop(),
                loss='mse',
                metrics=['mae'])

In [19]:
# # Compile the model
# model.compile(optimizer=optimizers.SGD(),
#                 loss='mse',
#                 metrics=['mae'])

In [20]:
#train the model (this takes a little while)
history = model.fit(X_train_scaled, y_train,
                    epochs=100, 
                    batch_size=32,
                    validation_data=(X_test_scaled, y_test))

Epoch 1/100


ValueError: in user code:

    File "/home/richard/miniconda3/envs/tf/lib/python3.11/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/richard/miniconda3/envs/tf/lib/python3.11/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/richard/miniconda3/envs/tf/lib/python3.11/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/richard/miniconda3/envs/tf/lib/python3.11/site-packages/keras/engine/training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "/home/richard/miniconda3/envs/tf/lib/python3.11/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/richard/miniconda3/envs/tf/lib/python3.11/site-packages/keras/

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(10,5), layout="constrained", sharex=True)

axs[0].plot(history.history['mae'])
axs[0].plot(history.history['val_mae'])
axs[0].set_title('Model MAE')
axs[0].set_ylabel('MAE')
axs[0].set_xlabel('Epoch')
# axs[0].set_ylim(0,0.01)
axs[0].legend(['Train', 'Test'], loc='upper left')

axs[1].plot(history.history['loss']) 
axs[1].plot(history.history['val_loss']) 
axs[1].set_title('Model loss') 
axs[1].set_ylabel('Loss') 
axs[1].set_xlabel('Epoch') 
axs[1].legend(['Train', 'Test'], loc='upper left') 

plt.show()

In [ ]:
# load JS visualization for notebook vis
shap.initjs()

In [ ]:
background = shap.sample(X_train_scaled, 100)

In [ ]:
explainer = shap.KernelExplainer(model.predict, background)

In [ ]:
shap_values = explainer.shap_values(X_test_scaled[:50], nsamples=50)

In [ ]:
shap.summary_plot(shap_values, X_test_scaled, feature_names=data.columns)

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[0], X_test_scaled[:100], feature_names=data.columns)

In [ ]:
y_pred_nn = model.predict(X_test_scaled).ravel()

In [ ]:
nn_r2 = r2_score(y_test, y_pred_nn)

nn_mae = mean_absolute_error(y_test, y_pred_nn)
nn_mse = mean_squared_error(y_test, y_pred_nn)
nn_rmse = np.sqrt(nn_mse)


In [ ]:
print("NN Model R2:", nn_r2)
print("NN Model MAE:", nn_mae)
print("NN Model MSE:", nn_mse)
print("NN Model RMSE:", nn_rmse)

## Random Forest

In [ ]:
# # gridsearch to give good challenge to neural net

# params = {
#     "n_estimators":[200,300,400,500],
#     "criterion":("squared_error", "absolute_error"),
#     "max_depth":[10,14,18]
#          } 

# rf = RandomForestRegressor()

# clf = GridSearchCV(rf, params, verbose=1)

In [ ]:
# #fitting the model
# clf.fit(X_train_scaled, y_train)

In [ ]:
# clf.best_params_

In [ ]:
#run this cell instead of the gridsearch to save alot of time.
clf = RandomForestRegressor(criterion="squared_error", max_depth=14, n_estimators=300)

clf.fit(X_train_scaled, y_train)

In [ ]:
# #this was from diff gridsearch
# clf = RandomForestRegressor(criterion="squared_error", max_depth=18, n_estimators=500)

# clf.fit(X_train_scaled, y_train)

In [ ]:
def sort_list(list1, list2): #function from geeksforgeeks.org
    zipped_pairs = zip(list2, list1)
    z = [x for _, x in sorted(zipped_pairs, reverse=True)]
    return z

In [ ]:
rf_feat_imp = clf.feature_importances_
feat_names = data.columns

ranked_feat_names = sort_list(feat_names, rf_feat_imp)

ranked_rf_feature_imp = sorted(rf_feat_imp, reverse=True)

fig, ax = plt.subplots(figsize=(18,4))
ax.bar(ranked_feat_names, ranked_rf_feature_imp)
plt.title("Feature Importance for Random Forest Regressor")
plt.show()

In [ ]:
#predictions
y_pred_rf = clf.predict(X_test_scaled)

In [ ]:
rf_r2 = r2_score(y_test, y_pred_rf)

rf_mae = mean_absolute_error(y_test, y_pred_rf)
rf_mse = mean_squared_error(y_test, y_pred_rf)
rf_rmse = np.sqrt(rf_mse)


In [ ]:
print("RF Model R2:", rf_r2)
print("RF Model MAE:", rf_mae)
print("RF Model MSE:", rf_mse)
print("RF Model RMSE:", rf_rmse)

## Volume Correction

In [ ]:
VC_pred = dt.VolumeCorrection(X_test.vol)

In [ ]:
VC_r2 = r2_score(y_test, VC_pred)

VC_mae = mean_absolute_error(y_test, VC_pred)
VC_mse = mean_squared_error(y_test, VC_pred)
VC_rmse = np.sqrt(VC_mse)

In [ ]:
print("VC Model R2:", VC_r2)
print("VC Model MAE:", VC_mae)
print("VC Model MSE:", VC_mse)
print("VC Model RMSE:", VC_rmse)

## Looking at Metrics and Predictions

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(8,8), layout="constrained")
models = ["Vol. Correction", "Rand. Forest", "Neural Net."]
r2_metric = [VC_r2, rf_r2, nn_r2]
mae_metric = [VC_mae, rf_mae, nn_mae]
mse_metric = [VC_mse, rf_mse, nn_mse]
rmse_metric = [VC_rmse, rf_rmse, nn_rmse]

axs[0][0].bar(models, r2_metric)
axs[0][1].bar(models, mae_metric)
axs[1][0].bar(models, mse_metric)
axs[1][1].bar(models, rmse_metric)

axs[0][0].set_title("R2 Score (Higher is Better)")
axs[0][1].set_title("MAE (Lower is Better)")
axs[1][0].set_title("MSE (Lower is Better)")
axs[1][1].set_title("RMSE (Lower is Better)")

plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12,4), layout="constrained")#, sharex=True, sharey=True)

im0 = axs[0].hist2d(y_test, VC_pred, bins=50, cmap="magma")
im1 = axs[1].hist2d(y_test, y_pred_rf, bins=50, cmap="cividis")
im2 = axs[2].hist2d(y_test, y_pred_nn, bins=50)

axs[0].plot([0,1],[0,1], "w--")
axs[1].plot([0,1],[0,1], "w--")
axs[2].plot([0,1],[0,1], "w--")

fig.colorbar(im0[3], ax=axs[0])
fig.colorbar(im1[3], ax=axs[1])
fig.colorbar(im2[3], ax=axs[2])

axs[0].set_xlabel("True Value")
axs[0].set_ylabel("Volume Correction Prediction")

axs[1].set_xlabel("True Value")
axs[1].set_ylabel("Random Forest Prediction")

axs[2].set_xlabel("True Value")
axs[2].set_ylabel("Neural Network Prediction")

plt.show()

In [ ]:
plt.plot(y_test, y_test, ".", label="Target")
plt.plot(y_test, VC_pred, ".", label="Volume Correction")
plt.plot(y_test, y_pred_rf, ".", label="Random Forest")
plt.plot(y_test, y_pred_nn, ".", label="Neural Network")
plt.xlabel("True Value")
plt.ylabel("Model Prediction")
plt.legend()
plt.show()

In [ ]:
#this plot is a bit of a mess
plt.plot(X_test.vol, y_test, ".", label="Target")
plt.plot(X_test.vol, VC_pred, ".", label="Volume Correction", alpha=0.5)
plt.plot(X_test.vol, y_pred_rf, ".", label="Random Forest", alpha=0.5)
plt.plot(X_test.vol, y_pred_nn, ".", label="Neural Network", alpha=0.5)
plt.xlabel("Local Volume")
plt.ylabel("Diffusion Parameter")
plt.legend()
plt.show()

In [ ]:
ind = 20
num = 1

plt.plot(X_test.vol[ind:ind+num], VC_pred[ind:ind+num], ".", label="Volume Correction")
plt.plot(X_test.vol[ind:ind+num], y_pred_rf[ind:ind+num], ".", label="Random Forest")
plt.plot(X_test.vol[ind:ind+num], y_pred_nn[ind:ind+num], ".", label="Neural Network")
plt.plot(X_test.vol[ind:ind+num], y_test[ind:ind+num], ".", label="Target")
# plt.xlim(0.12, 0.26)
# plt.ylim(0.3, 0.475)
plt.legend()
plt.show()